In [1]:
import os

import numpy as np

import tensorflow as tf
from tensorflow import keras



from RoadDataset import RoadDataset

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
#val_datasets_path = "../notebooks/datasets/validation/"
val_datasets_path = "../notebooks/datasets/train/"
sub_dirs = os.listdir(val_datasets_path)
val_dataset_paths = [[val_datasets_path + sub_dir + "/" + sub_dir + ".csv", val_datasets_path + sub_dir + "/frames/"] for sub_dir in sub_dirs]
val_dataset_paths

[['../notebooks/datasets/train/Sun Dec 27 18_09_33 2020/Sun Dec 27 18_09_33 2020.csv',
  '../notebooks/datasets/train/Sun Dec 27 18_09_33 2020/frames/'],
 ['../notebooks/datasets/train/Sun Dec 27 17_46_56 2020/Sun Dec 27 17_46_56 2020.csv',
  '../notebooks/datasets/train/Sun Dec 27 17_46_56 2020/frames/'],
 ['../notebooks/datasets/train/Sun Feb 21 21:50:50 2021/Sun Feb 21 21:50:50 2021.csv',
  '../notebooks/datasets/train/Sun Feb 21 21:50:50 2021/frames/'],
 ['../notebooks/datasets/train/Sun Dec 27 18_01_53 2020/Sun Dec 27 18_01_53 2020.csv',
  '../notebooks/datasets/train/Sun Dec 27 18_01_53 2020/frames/'],
 ['../notebooks/datasets/train/Wed Dec 30 17_34_37 2020/Wed Dec 30 17_34_37 2020.csv',
  '../notebooks/datasets/train/Wed Dec 30 17_34_37 2020/frames/'],
 ['../notebooks/datasets/train/Sun Jan 10 11:31:18 2021/Sun Jan 10 11:31:18 2021.csv',
  '../notebooks/datasets/train/Sun Jan 10 11:31:18 2021/frames/'],
 ['../notebooks/datasets/train/Sun Dec 27 18_25_24 2020/Sun Dec 27 18_25_24 

In [4]:
dataset_index = 28
column_names = ['filename','steering_wheel_angle','speed', 'blink_l', 'blink_r']
val_set = RoadDataset(val_dataset_paths[dataset_index][0], val_dataset_paths[dataset_index][1], column_names, batch_size=16)
val_set.clean()

Reading csv file: ../notebooks/datasets/train/Sun Jan 10 11:30:59 2021/Sun Jan 10 11:30:59 2021.csv
Loaded dataset with 31859 samples
Cleaning dataset. Starting with 31859 samples.
Removing 3475 rows for reason: Low speed (< 25 km/h)
Removing 760 rows for reason: High SWA (> +45/ < -45 deg)
Removing 270 rows for reason: Left blinker on
Removing 822 rows for reason: Right blinker on
26532 samples remaining.


In [5]:
stats = val_set.get_csv().describe()
stats = stats.transpose()
stats

,count,mean,std,min,25%,50%,75%,max
steering_wheel_angle,26532.0,-0.822223,9.568676,-44.700,-4.88,-0.940,2.630,44.890
speed,26532.0,70.161519,26.487415,25.048,50.84,65.782,77.531,150.474
blink_l,26532.0,0.000000,0.000000,0.000,0.00,0.000,0.000,0.000
blink_r,26532.0,0.000000,0.000000,0.000,0.00,0.000,0.000,0.000


In [6]:
#val_set.balance()
val_set.normalize()

In [7]:
stats = val_set.get_csv().describe()
stats = stats.transpose()
stats

,count,mean,std,min,25%,50%,75%,max
steering_wheel_angle,26532.0,-0.009136,0.106319,-0.496667,-0.054222,-0.010444,0.029222,0.498778
speed,26532.0,70.161519,26.487415,25.048000,50.840000,65.782000,77.531000,150.474000
blink_l,26532.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
blink_r,26532.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
#model_name = 'diypilot_v1'
#model_name = 'diypilot_v2_epoch_10'
#model_name = 'diypilot_v3_epoch_5'
#model_name = 'openpilot_replica_epoch_59'
#model_name = 'openpilot_replica_mae_epoch_100'
#model_name = 'diypilot_v9_small_FC_epoch_3'
#model_name = 'diypilot_v11_random_shifts_epoch_3'
#model_name = 'diypilot_v11_random_shifts_full_balance_epoch_3'
#model_name = 'diypilot_v11_full_balance_epoch_163'
#model_name = 'diypilot_v12_epoch_58'
#model_name = 'diypilot_v13_sum_min5_overrepr_epoch_10'
#model_name = 'diypilot_v14_smaller_crop_epoch_3'
#model_name = 'diypilot_v14_smaller_crop_wider_epoch_3'
#model_name = 'diypilot_v14_smaller_crop_wider_white_corners_epoch_10'
model_name = 'diypilot_v14_smaller_crop_wider_white_corners_random_shift_epoch_5'
loaded_model = keras.models.load_model('./trained_models/' + model_name + '.h5')
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 170, 848, 3)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 170, 282, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 82, 138, 16)       2368      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 41, 69, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 33, 32)        12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 16, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 6, 64)          5

In [9]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from Jetson.autopilot.record.AutopilotGUI import AutopilotGUI

In [10]:
gui = AutopilotGUI()

In [11]:
def show_frame(test_image_index):

    test_sample = val_set.get_frame(test_image_index)
    test_image = test_sample[0]
    test_image_cropped = val_set.get_cropped_frame(test_image_index, shift=0)[0]
    test_labels = test_sample[1]

    print("Test csv: " + str(test_labels))

    test_data = np.expand_dims(test_image_cropped, axis=0)
    predictions = loaded_model.predict(test_data)

    gui.set_frame(test_image)
    gui.set_show_overlay(True)
    gui.set_engaged(False)
    gui.set_recording(True)

    timestamp = test_labels['filename']
    gui.set_timestring(timestamp)

    velocity = test_labels['speed']
    gui.set_velocity(velocity)

    actual_swa = test_labels['steering_wheel_angle']*90
    gui.set_actual_swa(actual_swa)

    predicted_swa = predictions[0][0]*90
    
    swa_error = predicted_swa - actual_swa
    next_shift = swa_error / val_set.get_shift_multiplier()
    
    next_frame = val_set.get_cropped_frame(test_image_index+1, shift=next_shift)[0]
    
    
    print("Ground-truth: " +str(actual_swa))
    print("Prediction: " + str(predicted_swa))
    gui.set_predicted_swa(predicted_swa)

    gui.show_jupyter()

In [12]:
import ipywidgets as widgets
frame_slider = widgets.IntSlider(min=0, max=val_set.get_batch_size()*len(val_set)-1, continuous_update=False)

In [13]:
from ipywidgets import interact

interact(show_frame, test_image_index=frame_slider);

interactive(children=(IntSlider(value=0, continuous_update=False, description='test_image_index', max=26527), …

In [16]:
def sim_step(frame_idx, last_shift):
    test_sample = val_set.get_frame(frame_idx)
    test_image = test_sample[0]
    test_image_cropped = val_set.get_cropped_frame(frame_idx, shift=last_shift, offset=0)[0]
    test_image_cropped_unshifted = val_set.get_cropped_frame(frame_idx, shift=0, offset=0)[0]
    test_labels = test_sample[1]

    test_data = np.expand_dims(test_image_cropped, axis=0)
    predictions = loaded_model.predict(test_data)
    
    timestamp = test_labels['filename']
    gui.set_timestring(timestamp)

    actual_swa = test_labels['steering_wheel_angle']*90
    predicted_swa = predictions[0][0]*90
    
    swa_error = predicted_swa - actual_swa
    this_shift = -swa_error #/ val_set.get_shift_multiplier()
    
    next_frame = val_set.get_cropped_frame(frame_idx+1, shift=this_shift + last_shift, offset=0)[0]
    
    #gui.set_frame(next_frame)
    #gui.set_show_overlay(True)
    #gui.show_jupyter()
    
    return this_shift, next_frame, test_image_cropped_unshifted

In [17]:
import cv2
import time
fourcc = cv2.VideoWriter_fourcc(*'XVID')
filename = model_name + "__" + str(time.time())
print(filename)
out = cv2.VideoWriter(filename + '.avi',fourcc, 15.0, (848,340))

total_shift = 0
aborts = 0
for frame_idx in range(5000):
    shift, frame, unshifted_frame = sim_step(frame_idx, total_shift)
    total_shift += shift
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    video_frame = np.vstack((unshifted_frame, frame))
    out.write(video_frame.astype(np.uint8))
    
    print(frame_idx, total_shift)
    
    if (abs(total_shift) > 150):
        
        # abort, insert 1s black in video to visualize failure and restart with 0 shift
        dummy_frame = np.zeros((340,848,3), np.uint8)
        
        for i in range(15):
            out.write(dummy_frame)
            
        total_shift = 0
        aborts += 1
    
out.release()

print("Aborts: " + str(aborts))

diypilot_v14_smaller_crop_wider_white_corners_random_shift_epoch_5__1619974806.2363703
0 -1.510305204540491
1 -7.0484374673664565
2 -13.068631431609392
3 -13.72949937224388
4 -17.41389094531536
5 -15.814595459550617
6 -21.917472539991138
7 -26.099957032650707
8 -38.478281869143245
9 -48.49946465954184
10 -61.43788838252425
11 -54.47052017971873
12 -44.74930317088962
13 -35.68563913926482
14 -30.445408367961644
15 -28.132575216591356
16 -24.204593748152252
17 -29.99607810169458
18 -39.896931301057336
19 -42.920204994976515
20 -31.330274910032742
21 -21.505884769260874
22 -12.129666254222384
23 -7.366509301364412
24 -1.6679768815636535
25 1.5592448955774407
26 6.412931081652651
27 10.543436720371256
28 11.524501384049664
29 16.92458602860571
30 24.130376697331677
31 28.585423202067624
32 31.691310165673503
33 27.94773099541665
34 23.395658733844765
35 23.664856505990038
36 17.262775503397
37 10.817735570669186
38 4.654912306070339
39 -0.30147681295870576
40 3.565464470386517
41 10.202006

TypeError: int() argument must be a string, a bytes-like object or a number, not 'JpegImageFile'